In [1]:
!pip install ibmos2spark

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20201014072227-0000
KERNEL_ID = f0d60a44-fa65-4e62-aa88-0e7924aff9ea


In [2]:
import ibmos2spark
# @hidden_cell
credentials = {
    'endpoint': 'https://s3.eu-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-5119afa5-9f36-4561-8de3-537f498c07d2',
    'iam_service_endpoint': 'https://iam.cloud.ibm.com/oidc/token',
    'api_key': 'UdCSHHwYJp9QLwhI_Gt3F7rxwpCcfZ48WNZKvwQlD1Hk'
}

configuration_name = 'os_ab26667fc49e46aaa28ad8a08f0a62c0_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df_data_1 = spark.read.parquet(cos.url('hmp.parquet', 'scalablemachinelearningonbigdatau-donotdelete-pr-zvgztoifq8gdmt'))
df_data_1.show()


+---+---+---+--------------------+-----------+
|  x|  y|  z|              source|      class|
+---+---+---+--------------------+-----------+
| 22| 49| 35|Accelerometer-201...|Brush_teeth|
| 22| 49| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|
| 21| 52| 34|Accelerometer-201...|Brush_teeth|
| 22| 51| 34|Accelerometer-201...|Brush_teeth|
| 20| 50| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 34|Accelerometer-201...|Brush_teeth|
| 22| 50| 34|Accelerometer-201...|Brush_teeth|
| 22| 51| 35|Accelerometer-201...|Brush_teeth|
| 21| 51| 33|Accelerometer-201...|Brush_teeth|
| 20| 50| 34|Accelerometer-201...|Brush_teeth|
| 21| 49| 33|Accelerometer-201...|Brush_teeth|
| 21| 49| 33|Accelerometer-201...|Brush_teeth|
| 20| 51| 35|Accelerometer-201...|Brush_teeth|
| 18| 49| 34|Accelerometer-201...|Brush_teeth|
| 19| 48| 34|Accelerometer-201...|Brush_teeth|
| 16| 53| 34|Accelerometer-201...|Brush_teeth|
| 18| 52| 35|

In [3]:
# copying df_data_1 to df for simple usage
df = df_data_1

In [4]:
# splitting in train and test data sets in 80:20
splits = df.randomSplit([0.8, 0.2])
df_train = splits[0]
df_test = splits[1]

In [5]:
# preprocessing steps
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer


indexer = StringIndexer(inputCol="class", outputCol="label")

vectorAssembler = VectorAssembler(inputCols=["x","y","z"],
                                  outputCol="features")

normalizer = Normalizer(inputCol="features", outputCol="features_norm", p=1.0)

In [6]:
# Logistic Regression
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

In [7]:
# Creating pipeline
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[indexer, vectorAssembler, normalizer,lr])

In [8]:
# training the model
model = pipeline.fit(df_train)

In [9]:
# predicting on training data
prediction = model.transform(df_train)

In [10]:
# evaluation using MulticlassClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy") .setPredictionCol("prediction").setLabelCol("label")
    
binEval.evaluate(prediction)

0.2068701648166835

In [11]:
# Prediction on testdata
prediction = model.transform(df_test)

In [12]:
# evaluating test prediction
binEval.evaluate(prediction)

0.20553866033931537